In [1]:
import gym
import ray
from ray import tune
from ray.rllib.agents.ppo import PPOTrainer
from scouting_gym.tasks.scouting_discrete_task import ScoutingDiscreteTask


Instructions for updating:
non-resource variables are not supported in the long term
None


In [2]:
env = gym.make('Scouting-v0')

print(env.observation_space)

[ERROR] [1611727512.596370, 0.001000]: NOT Initialising Simulation Physics Parameters
[WARN] [1611727512.604854, 0.032000]: Start Init ControllersConnection
[WARN] [1611727512.606105, 0.032000]: END Init ControllersConnection


Box(0.0, 8.0, (12,), float32)


In [3]:
env.reset()
for _ in range(20):
    #obs, reward, done, _ = env.step(1)
    obs, reward, done, _ = env.step([0.6, ])
print(obs.min())
print(obs.max())

1.2628748416900635
7.99


In [4]:
print(obs)

[2.31133652 2.18991303 2.19176579 2.31844544 3.04565477 5.49906635
 2.58944726 1.55777252 1.29146457 1.26287484 7.24449328 7.99      ]


In [5]:
ray.init()

2021-01-27 07:05:16,366	INFO services.py:1171 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.178.60',
 'raylet_ip_address': '192.168.178.60',
 'redis_address': '192.168.178.60:6379',
 'object_store_address': '/tmp/ray/session_2021-01-27_07-05-15_875136_2534000/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-01-27_07-05-15_875136_2534000/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-01-27_07-05-15_875136_2534000',
 'metrics_export_port': 62359,
 'node_id': 'dad1d6c56212a0f07ce49f7e02f7203259f40839'}

In [6]:
config = {
    "env": ScoutingDiscreteTask,  # or "corridor" if registered above
    "env_config": {
        "corridor_length": 5,
    },
    # Use GPUs iff `RLLIB_NUM_GPUS` env var set to > 0.

    "num_gpus": 1,
    "num_workers": 1,  # parallelism
    "model": {
        "fcnet_hiddens": [128, 128, ],
        #"fcnet_hiddens": tune.grid_search([[64, 64, ], [128, 128, ], [256, 256, ]])
    }
    #"model": {
    #    "dim": 40,
    #    "conv_filters": [[16, [3, 3], 2], [32, [3, 3], 2], [64, [3, 3], 2], [512, [5, 16], 1]]
    #}
    #"model": {
    #    "use_lstm": True,
    #    # Max seq len for training the LSTM, defaults to 20.
    #    "max_seq_len": 20,
    #    # Size of the LSTM cell.
    #    "lstm_cell_size": 256
    #}
}

stop = {
    "episodes_total": 2500,
}

In [7]:
def train(stop_criteria, config, restorepath):
    """
    Train an RLlib PPO agent using tune until any of the configured stopping criteria is met.
    :param stop_criteria: Dict with stopping criteria.
        See https://docs.ray.io/en/latest/tune/api_docs/execution.html#tune-run
    :return: Return the path to the saved agent (checkpoint) and tune's ExperimentAnalysis object
        See https://docs.ray.io/en/latest/tune/api_docs/analysis.html#experimentanalysis-tune-experimentanalysis
    """
    analysis = ray.tune.run(PPOTrainer, config=config,
                            stop=stop_criteria,
                            checkpoint_freq=1,
                            checkpoint_at_end=True, restore=restorepath)
    # list of lists: one list per checkpoint; each checkpoint list contains 1st the path, 2nd the metric value
    checkpoints = analysis.get_trial_checkpoints_paths(trial=analysis.get_best_trial('episode_reward_mean', mode='max'),
                                                       metric='episode_reward_mean',
                                                       )
    # retriev the checkpoint path; we only have a single checkpoint, so take the first one
    checkpoint_path = checkpoints[0][0]
    return checkpoint_path, analysis

def load(checkpoint_path, config):
    """
    Load a trained RLlib agent from the specified path. Call this before testing a trained agent.
    :param path: Path pointing to the agent's saved checkpoint (only used for RLlib agents)
    """
    agent = PPOTrainer(config=config)
    agent.restore(checkpoint_path)
    return agent

def test(agent, env):
    """Test trained agent for a single episode. Return the episode reward"""
    # instantiate env class

    # run until episode ends
    episode_reward = 0
    done = False
    obs = env.reset()
    while not done:
        action = agent.compute_action(obs)
        obs, reward, done, info = env.step(action)
        episode_reward += reward

    return episode_reward

In [ ]:
checkpoint_path, analysis = train(stop_criteria=stop,
                                  config=config,
                                  restorepath='/home/dschori/ray_results/PPO_2021-01-26_21-34-32/PPO_ScoutingDiscreteTask_e5473_00000_0_2021-01-26_21-34-32/checkpoint_100/checkpoint-100')

Trial name,status,loc
PPO_ScoutingDiscreteTask_a156d_00000,RUNNING,


(pid=2534259) WARNING:tensorflow:From /home/dschori/.local/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=2534259) Instructions for updating:
(pid=2534259) non-resource variables are not supported in the long term
(pid=2534259) 2021-01-27 07:05:21,147	INFO trainer.py:591 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
(pid=2534259) 2021-01-27 07:05:21,147	INFO trainer.py:616 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=2534265) WARNING:tensorflow:From /home/dschori/.local/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=2534265) Instructions for updating:
(pid=2534265) non-reso

(pid=2534259) None
(pid=2534265) None
Result for PPO_ScoutingDiscreteTask_a156d_00000:
  custom_metrics: {}
  date: 2021-01-27_07-08-11
  done: false
  episode_len_mean: 487.375
  episode_reward_max: 116.02267329880763
  episode_reward_mean: -67.3726155448889
  episode_reward_min: -107.9129123412074
  episodes_this_iter: 8
  episodes_total: 1343
  experiment_id: 8b40e97173b34a5286d39a888fffb61b
  hostname: workstation
  info:
    learner:
      default_policy:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 4.999999873689376e-05
        entropy: 0.8472505807876587
        entropy_coeff: 0.0
        kl: 0.010873379185795784
        model: {}
        policy_loss: -0.01301891915500164
        total_loss: 460.656982421875
        vf_explained_var: 0.3018514811992645
        vf_loss: 460.66790771484375
    num_steps_sampled: 404000
    num_steps_trained: 404000
  iterations_since_restore: 1
  node_ip: 192.168.178.60
  num_healthy_workers: 1
  off_policy_estimator: {}
  perf:
    c

Trial name,status,loc
PPO_ScoutingDiscreteTask_a156d_00000,RUNNING,


Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_ScoutingDiscreteTask_a156d_00000,RUNNING,192.168.178.60:2534259,101,15295.4,404000,-67.3726,116.023,-107.913,487.375


...

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_ScoutingDiscreteTask_a156d_00000,RUNNING,192.168.178.60:2534259,124,18918.5,496000,-57.6866,117.531,-109.413,308.77


Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_ScoutingDiscreteTask_a156d_00000,RUNNING,192.168.178.60:2534259,125,19082.1,500000,-55.5984,118.545,-109.413,296.35


Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_ScoutingDiscreteTask_a156d_00000,RUNNING,192.168.178.60:2534259,126,19236.6,504000,-55.4985,118.545,-109.413,312.21


Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_ScoutingDiscreteTask_a156d_00000,RUNNING,192.168.178.60:2534259,127,19394.4,508000,-52.8275,118.545,-102.127,326.6


Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_ScoutingDiscreteTask_a156d_00000,RUNNING,192.168.178.60:2534259,128,19556.3,512000,-55.0419,118.545,-102.127,331.52


In [13]:
checkpoint_path

'/home/dschori/ray_results/PPO_2021-01-27_06-42-22/PPO_ScoutingDiscreteTask_6db9c_00000_0_2021-01-27_06-42-22/checkpoint_1/checkpoint-1'

In [14]:
agent = load(checkpoint_path=checkpoint_path, config=config)

episode_reward = test(agent=agent, env=env)
episode_reward

2021-01-27 06:47:08,421	INFO trainer.py:591 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
2021-01-27 06:47:08,422	INFO trainer.py:616 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=2483286) WARNING:tensorflow:From /home/dschori/.local/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=2483286) Instructions for updating:
(pid=2483286) non-resource variables are not supported in the long term
(pid=2483286) [ERROR] [1611726431.556651, 0.000000]: NOT Initialising Simulation Physics Parameters
(pid=2483286) [WARN] [1611726431.559983, 0.000000]: Start Init ControllersConnection
(pid=2483286) [WARN] [1611726431.560941, 0.000000]: END Init ControllersConnection
2021-01-27 06:47:17,369	WARNING util.py:43 -- Install gputil for GPU system mo

(pid=2483286) None


-93.38296034454262